In [ ]:
import os, csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from scipy import stats
from itertools import combinations

from sklearn.model_selection import GroupShuffleSplit, GroupKFold, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

destination = ''

dataFile = os.path.join(destination, '')

data = pd.read_csv(dataFile, sep = ",")
df = pd.DataFrame(data)
df = df.fillna(0)

input_list = ['T1', 'T2', 'TC', 'FL']

for r in range(1, len(input_list) + 1):
    for combination in combinations(input_list, r):
        
        input_mod = list(combination)
        print(input_mod)

        data = df[input_mod]
        target = df['CD']


        param_grid = {
            'n_estimators': [50, 100, 200, 300],
             'max_depth': [ 10, 20, 30, 40],
            'min_samples_split': [2,3,4, 5]
        }

        # Initialize GroupShuffleSplit with 3 splits
        gss = GroupKFold(n_splits=5)

        # Initialize and train the Random Forest Regressor
        model = RandomForestRegressor(random_state= 34)

        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=gss, scoring='neg_mean_squared_error')
        
        grid_search.fit(data, target, groups=df['InstanceUID'])

        # Print the best parameters and score
        print("Best parameters:", grid_search.best_params_)
        print("Best score:", grid_search.best_score_)
        
        # Get the best estimator
        best_rf = grid_search.best_estimator_
        
        